# **Introduction**
  ### **Hey there, movie buffs! Tired of scrolling endlessly to find something good to watch?   Ever feel like the OTT platform just gets you?**

  ### **Well, buckle up because we're about to crack the code on recommendations!   Join us at 07:07 AM to dive into the world of how these platforms know your taste in movies.  We'll be spilling the tea on those eerily perfect suggestions.**

**Link to the datasource: https://drive.google.com/drive/folders/1lgXm5Lyqjdkisn1G9c-MqaeLT4boNIzg?usp=sharing**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<hr>

# **Building the Recommendation Engine 🎞️**

**Ever scrolled through Amazon and felt like they can read your mind? That's the magic of recommendation systems!  Basically, it's a super-powered filter that picks up on what you search for or buy, then uses that info to suggest similar things you might like. It's like having a personal shopping buddy who remembers your interests and whispers "Hey, check this out!" whenever you're browsing. Pretty cool, right?**

**Ever notice how YouTube seems to know exactly what you want to watch next? That's because it's got a recommendation system working behind the scenes, like a super-smart friend suggesting videos based on what you've watched before. It's the same deal with Netflix – it learns your taste in movies and genres, then whispers in your ear (well, the recommendation bar) with suggestions you might love.**

**This recommendation system magic isn't just for entertainment. It's everywhere!  Scrolling through Facebook or Instagram? Boom, recommendations for new friends and accounts to follow pop up.  Shopping online?  Amazon, BigBasket, and other sites use your past searches and purchases to show you ads and products that might catch your eye. It's like having a personal shopping buddy who remembers what you like and says "Hey, check this out!"**

**So, the next time you see those eerily perfect recommendations, remember – it's not magic, it's just a clever system that helps you discover new things and maybe even find that perfect product (or next binge-worthy video).**

<hr>

**Types of Recommendation System**
  * **`Demographic Filtering`: Imagine you walk into a movie store blindfolded. The salesperson, armed with only your age and maybe your favorite color, recommends the "blockbuster hits" everyone's raving about. That's kind of how demographic filtering works. It uses broad categories like age, gender, or location to suggest movies that are generally popular with similar groups. While it can be a good starting point, it doesn't account for your unique tastes. It's like getting a generic recommendation instead of a friend suggesting a hidden gem they know you'll love.**


  * **`Content Based Filtering` : Ever notice how after watching a funny cat video on YouTube, you get bombarded with suggestions for more feline frolics? That's content-based filtering at work! This system is like a detective, looking at the clues – things like genre, director, or actors for movies – to find items that are similar to what you liked before. The idea is that if you enjoyed something, you'd probably enjoy something else with similar characteristics. It's a good way to discover hidden gems within a category you already love, but it might not introduce you to entirely new things outside your comfort zone.**

  * **`Collabrative Filtering` - Imagine you're at a party and hit it off with someone who has amazing taste in movies. They rave about this hidden gem you've never heard of, and you know you gotta check it out because you trusted their other picks. That's the magic of collaborative filtering! Unlike content-based systems that focus on the movie itself, this one is all about finding users with similar tastes to you. It's like having a secret network of movie buddies who recommend things they know you'll love, based on what they've enjoyed themselves. Pretty cool, right?**

In [ ]:
data_location = "/content/drive/MyDrive/Datasets/Netflix/Data for Files/combinedNetflixData.txt"

<hr>

# **Step 1 - Importing of Libraries**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

<hr>

# **Step 2 - Data Loading**

In [ ]:
df = pd.read_csv(data_location, names = ["CustID", "Ratings"], usecols = [0, 1], header = None)

In [ ]:
df.head()

CustID  Ratings
0       1:      NaN
1  1488844      3.0
2   822109      5.0
3   885013      4.0
4    30878      4.0

### **Final Agenda:**
  **We need to differentiate the custID, Ratings and MovieID columns**

**Looking at the data, we can observe that the whatever is in `x:` is our movie
ids, and below that whatever ratings has been given they correspond to that particular movie `x`**

<hr>

###**Shape Inspection**

In [ ]:
print(f"Rows: {df.shape[0]} and Columns: {df.shape[1]}")

Rows: 24058263 and Columns: 2


<hr>

### **Fetch some relevant information**

* **How many movies we are dealing with in total?**

In [ ]:
total_count_of_movies = df.isnull().sum()["Ratings"]

In [ ]:
print("The total movies are: ", total_count_of_movies)

The total movies are:  4499


<hr>

* **Total Customer Count**

In [ ]:
customer_count = df["CustID"].nunique() - total_count_of_movies

print("Total Number of Customer: ", customer_count)

Total Number of Customer:  470758


<hr>

* **How many ratings has been given**

In [ ]:
total_ratings_count = df["CustID"].count() - total_count_of_movies

In [ ]:
print("Total Ratings: ",total_ratings_count)

Total Ratings:  24053764


<hr>

### **Division of Ratings**

In [ ]:
df["Ratings"].value_counts()

Ratings
4.0    8085741
3.0    6904181
5.0    5506583
2.0    2439073
1.0    1118186
Name: count, dtype: int64

In [ ]:
df.head(2)

CustID  Ratings
0       1:      NaN
1  1488844      3.0

<hr>

# **Step 3 - Segregation of data**

* **Step 1 - Create a copy of original data**

In [ ]:
df_copy = df.copy()

* **Step 2 - Assign the variables**

In [ ]:
# This variable will keep the track of the current movie
curr_movie = None

# That will hold the data for the mapping
movie_ids = []

<hr>

* **Step 3 - Implementation of Logic**

In [ ]:
# Loop for iterating over CustID column, to replace ":" with numbers
for cust_id in df_copy["CustID"]:
  # We will put the condition for checking the "x:"
  if ":" in cust_id:
    # We will try to replace : with nothing so that we can get a particular integer values
    curr_movie = int(cust_id.replace(":", ""))
  # We will take this movie and map with the data
  movie_ids.append(curr_movie)

# This lines will help us mapping data to ratings
df_copy["MovieID"] = movie_ids

# Removal of the unneccsary data
df_copy = df_copy[df_copy["Ratings"].notna()]

In [ ]:
df_copy.tail()

CustID  Ratings  MovieID
24058258  2591364      2.0     4499
24058259  1791000      2.0     4499
24058260   512536      5.0     4499
24058261   988963      3.0     4499
24058262  1704416      3.0     4499

<hr>

* **Another Apporach**

In [ ]:
# Creating the duplicates
temp = df.copy()

temp['movie_id'] = temp['CustID'].where(temp['Ratings'].isna()).ffill()

# Dropping the irrelevant columns
temp=temp.dropna(subset=['Ratings'])

# Strings splits
temp['movie_id'] = temp['movie_id'].astype(str).str.replace(':', '', regex=False)

In [ ]:
temp.head()

CustID  Ratings movie_id
1  1488844      3.0        1
2   822109      5.0        1
3   885013      4.0        1
4    30878      4.0        1
5   823519      3.0        1

<hr>

# **Step 4 - Preparation for Collabrative Filtering**

* **We cannot remove duplicates for any column since the requirement is completely based on the ratings of the users given, hence we will go ahead with the benchmarks**

* **Here there is a need of two benchmarks in the dataset**
  * **Customers who are not frequently giving rating (non active users or maybe fake or dummy users). These are users that watch movies but don't give ratings, so we can remove them**
  * **Those movies that has less ratings are possibly not much popular so will not recommend and remove them from the list**

<hr>

### **Benchmark 1 - Movies Part**

In [ ]:
df = df_copy

In [ ]:
df.head()

CustID  Ratings  MovieID
1  1488844      3.0        1
2   822109      5.0        1
3   885013      4.0        1
4    30878      4.0        1
5   823519      3.0        1

In [ ]:
movie_list = df.groupby("MovieID")["Ratings"].agg(["count"])

In [ ]:
movie_list

count
MovieID       
1          547
2          145
3         2012
4          142
5         1140
...        ...
4495       614
4496      9519
4497       714
4498       269
4499       428

[4499 rows x 1 columns]

* **Threshold for setting the movie as good movie or to be considered**

In [ ]:
movie_list["count"].quantile(0.7)

1798.6

In [ ]:
benchmark_movie = round(movie_list["count"].quantile(0.7), 0)

* **We can use this benchmark to decide which movie has to be there and which one to be dropped**

* **Any movie that is having less than 1800 ratings will be dropped, only movie having a lot of rating to be there in the list**

In [ ]:
drop_movie_list = movie_list[movie_list["count"] < benchmark_movie].index

In [ ]:
len(drop_movie_list)

3149

<hr>

### **Benchmark 2 - User Part**

In [ ]:
cust_list = df.groupby("CustID")["Ratings"].agg(["count"])

In [ ]:
cust_list

count
CustID        
10          49
1000004      1
1000027     30
1000033    101
1000035     20
...        ...
999964      48
999972      35
999977      14
999984      38
999988      28

[470758 rows x 1 columns]

* **Threshold for setting the movie as good movie or to be considered**

In [ ]:
cust_list["count"].quantile(0.7)

52.0

In [ ]:
benchmark_users = round(cust_list["count"].quantile(0.7), 0)

In [ ]:
drop_cust_list = cust_list[cust_list["count"] < benchmark_users].index

In [ ]:
len(drop_cust_list)

327300

In [ ]:
470758 - 327300

143458

<hr>

### **Part 3 - Removal of data**

In [ ]:
df.columns

Index(['CustID', 'Ratings', 'MovieID'], dtype='object')

In [ ]:
(df["CustID"].unique())

array(['1488844', '822109', '885013', ..., '1017887', '72311', '594210'],
      dtype=object)

* **To remove the movies and customer from the existing data using the drop_list that we have created**

In [ ]:
df = df[~df["MovieID"].isin(drop_movie_list)]  # Data - > isin(check) -> True -> False (remove)

df = df[~df["CustID"].isin(drop_cust_list)]

In [ ]:
print(f"Rows: {df.shape[0]} and Columns: {df.shape[1]}")

Rows: 17337458 and Columns: 3


<hr>

# **Step 5 - Working with recommendation**

### **Load our secondary data**

In [ ]:
movies_df = pd.read_csv("/content/drive/MyDrive/Datasets/Netflix/Data for Files/NetflixMovieData.csv", names = ["MovieID", "Year", "Name"], usecols = [0, 1, 2], header = None)

In [ ]:
movies_df.head()

MovieID    Year                          Name
0        1  2003.0               Dinosaur Planet
1        2  2004.0    Isle of Man TT 2004 Review
2        3  1997.0                     Character
3        4  1994.0  Paula Abdul's Get Up & Dance
4        5  2004.0      The Rise and Fall of ECW

### **Recommendation System with SVD**

**Package: scikit-surprise**

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357280 sha256=3c62d77166a65fb27b4d28886ddbd8a32d6bc58cad647ba3c3731a96a00eca32
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


### **Import some elements from SVD**

In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

### **Read the data**

In [ ]:
reader = Reader()

In [ ]:
data = Dataset.load_from_df(df[["CustID", "MovieID", "Ratings"]][:100000], reader)

<hr>

### **Model Building**

In [ ]:
model = SVD()

In [ ]:
cross_validate(model, data, measures = ["RMSE"], cv = 4)

{'test_rmse': array([0.99701596, 0.99803998, 0.99327303, 1.00060991]),
 'fit_time': (1.811328649520874,
  1.4967734813690186,
  2.223896026611328,
  2.017756462097168),
 'test_time': (0.13796019554138184,
  0.14218926429748535,
  0.21158981323242188,
  0.23064970970153809)}

<hr>

### **Creating filter for recommendation**

In [ ]:
df["CustID"].dtype

dtype('O')

In [ ]:
data_1331154 = df[(df["CustID"] == "1331154") & (df["Ratings"] == 5.0)]

In [ ]:
data_1331154

CustID  Ratings  MovieID
458308    1331154      5.0      143
1184450   1331154      5.0      270
1991774   1331154      5.0      361
2369367   1331154      5.0      457
2600328   1331154      5.0      482
3417458   1331154      5.0      658
4029215   1331154      5.0      763
5646194   1331154      5.0     1144
7075510   1331154      5.0     1425
7423467   1331154      5.0     1476
7468997   1331154      5.0     1495
8184220   1331154      5.0     1642
8257180   1331154      5.0     1650
8789450   1331154      5.0     1754
10165725  1331154      5.0     1974
10919877  1331154      5.0     2128
11519840  1331154      5.0     2192
12202244  1331154      5.0     2372
12746620  1331154      5.0     2452
14167769  1331154      5.0     2743
14525287  1331154      5.0     2795
15104095  1331154      5.0     2913
17319044  1331154      5.0     3333
17679853  1331154      5.0     3379
19254925  1331154      5.0     3650
19555314  1331154      5.0     3730
19992284  1331154      5.0     3825
20774457  1331154      5.0     3925
21174535  1331154      5.0     3962
21317452  1331154      5.0     3966
22718437  1331154      5.0     4306

In [ ]:
movies_df

MovieID    Year                                               Name
0            1  2003.0                                    Dinosaur Planet
1            2  2004.0                         Isle of Man TT 2004 Review
2            3  1997.0                                          Character
3            4  1994.0                       Paula Abdul's Get Up & Dance
4            5  2004.0                           The Rise and Fall of ECW
...        ...     ...                                                ...
17764    17766  2002.0  Where the Wild Things Are and Other Maurice Se...
17765    17767  2004.0                  Fidel Castro: American Experience
17766    17768  2000.0                                              Epoch
17767    17769  2003.0                                        The Company
17768    17770  2003.0                                       Alien Hunter

[17769 rows x 3 columns]

In [ ]:
df.head()

CustID  Ratings  MovieID
696   712664      5.0        3
697  1331154      4.0        3
698  2632461      3.0        3
699    44937      5.0        3
700   656399      4.0        3

In [ ]:
list_of_infinite_possibility = movies_df.copy()

In [ ]:
list_of_infinite_possibility.reset_index(inplace = True)

**These all are the movie that a user can possibly see. Now we will use this list and the data for framing the recommendation**

In [ ]:
list_of_infinite_possibility = list_of_infinite_possibility[~list_of_infinite_possibility["MovieID"].isin(drop_movie_list)]

### **Make Recommendation**

In [ ]:
list_of_infinite_possibility["Estimate Score"] = list_of_infinite_possibility["MovieID"].apply(lambda x : model.predict("44937", x).est)

In [ ]:
list_of_infinite_possibility

index  MovieID    Year                                Name  \
27        27       28  2002.0                     Lilo and Stitch   
17        17       18  1994.0                    Immortal Beloved   
29        29       30  2003.0              Something's Gotta Give   
2          2        3  1997.0                           Character   
12889  12889    12891  2003.0                        24: Season 3   
...      ...      ...     ...                                 ...   
8027    8027     8029  2004.0  Dora the Explorer: Catch the Stars   
7          7        8  2004.0          What the #$*! Do We Know!?   
15        15       16  1996.0                           Screamers   
16        16       17  2005.0                           7 Seconds   
25        25       26  2004.0                     Never Die Alone   

       Estimate Score  
27           3.929081  
17           3.898559  
29           3.895165  
2            4.334268  
12889        3.711754  
...               ...  
8027         3.711754  
7            3.029140  
15           3.270429  
16           2.840501  
25           2.425794  

[14620 rows x 5 columns]

In [ ]:
list_of_infinite_possibility = list_of_infinite_possibility.sort_values('Estimate Score', ascending=False)

In [ ]:
list_of_infinite_possibility

index  MovieID    Year                        Name  Estimate Score
2          2        3  1997.0                   Character        4.334268
27        27       28  2002.0             Lilo and Stitch        3.929081
17        17       18  1994.0            Immortal Beloved        3.898559
29        29       30  2003.0      Something's Gotta Give        3.895165
57        57       58  1996.0                 Dragonheart        3.711754
...      ...      ...     ...                         ...             ...
17760  17760    17762  1997.0                     Gattaca        3.711754
15        15       16  1996.0                   Screamers        3.270429
7          7        8  2004.0  What the #$*! Do We Know!?        3.029140
16        16       17  2005.0                   7 Seconds        2.840501
25        25       26  2004.0             Never Die Alone        2.425794

[14620 rows x 5 columns]